In [1]:
import mne
from glob import glob
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

In [2]:
def read_data(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.pick_types(meg=False, eeg=True, eog=False, ecg=False) # Selecting EEG, EOG and ECG channels
    # Select a specific channel
    channel_to_keep = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE', 'EEG A2-A1']  
    # Replace with the name of the channel you want to keep
    raw.pick_channels(channel_to_keep)
    raw.set_eeg_reference()
    raw.filter(l_freq=30,h_freq=100)#1-4=delta, 4-8=theta, 8-12=alpha, 12-30=beta, 30-100=gamma
    epochs=mne.make_fixed_length_epochs(raw,duration=15,overlap=1)
    epochs=epochs.get_data()
    scaler = StandardScaler()
    data = scaler.fit_transform(epochs.reshape(-1,epochs.shape[-1])).reshape(epochs.shape)
    return data #trials,channel,length

In [3]:
all_files_path=glob('../data/*.edf')
print(len(all_files_path))
healthy_file_path=[i for i in all_files_path if  'H' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  'M' in i.split('\\')[1]]
print(len(healthy_file_path),len(patient_file_path))


162
76 86


In [4]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]

In [5]:
len(control_epochs_array)

76

In [6]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]

data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups_list)

data_array=np.moveaxis(data_array,1,2)
X=data_array
y=label_array

In [7]:
print(len(y),len(X))

4638 4638


In [8]:
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(3840,20)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])
    return model

In [9]:
model = cnnmodel()
model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])
history = model.fit(X,y,epochs=25,batch_size=25)

Epoch 1/25
186/186 [==============================] - 21s 74ms/step - loss: 0.5284 - Accuracy: 0.7370 - precision: 0.6794 - recall: 0.9608 - auc: 0.8235
Epoch 2/25
186/186 [==============================] - 14s 74ms/step - loss: 0.2268 - Accuracy: 0.9041 - precision: 0.8757 - recall: 0.9560 - auc: 0.9703
Epoch 3/25
186/186 [==============================] - 13s 71ms/step - loss: 0.1642 - Accuracy: 0.9273 - precision: 0.9125 - recall: 0.9556 - auc: 0.9844
Epoch 4/25
186/186 [==============================] - 13s 72ms/step - loss: 0.1422 - Accuracy: 0.9405 - precision: 0.9317 - recall: 0.9588 - auc: 0.9883
Epoch 5/25
186/186 [==============================] - 14s 77ms/step - loss: 0.1246 - Accuracy: 0.9515 - precision: 0.9490 - recall: 0.9608 - auc: 0.9906
Epoch 6/25
186/186 [==============================] - 15s 82ms/step - loss: 0.1029 - Accuracy: 0.9588 - precision: 0.9568 - recall: 0.9665 - auc: 0.9933
Epoch 7/25
186/186 [==============================] - 15s 79ms/step - loss: 0.0940

In [10]:
print("training accuracy",np.mean(history.history['Accuracy']))

training accuracy 0.958956446647644


In [11]:
test_all_files_path=glob('../test/*.edf')
print(len(test_all_files_path))
test_healthy_file_path=[i for i in test_all_files_path if  'H' in i.split('\\')[1]]
test_patient_file_path=[i for i in test_all_files_path if  'M' in i.split('\\')[1]]

18


In [12]:
%%capture
test_control_epochs_array=[read_data(subject) for subject in test_healthy_file_path]
test_patients_epochs_array=[read_data(subject) for subject in test_patient_file_path]

In [13]:
test_control_epochs_labels=[len(i)*[0] for i in test_control_epochs_array]
test_patients_epochs_labels=[len(i)*[1] for i in test_patients_epochs_array]

test_data_list=test_control_epochs_array+test_patients_epochs_array
test_label_list=test_control_epochs_labels+test_patients_epochs_labels

test_data_array=np.vstack(test_data_list)
test_label_array=np.hstack(test_label_list)
test_data_array=np.moveaxis(test_data_array,1,2)

In [14]:
model.evaluate(test_data_array,test_label_array)

17/17 [==============================] - 2s 40ms/step - loss: 0.0877 - Accuracy: 0.9761 - precision: 0.9823 - recall: 0.9720 - auc: 0.9949  


[0.08773865550756454,
 0.9760589599609375,
 0.982332170009613,
 0.9720279574394226,
 0.9949389696121216]